In [57]:
# Librerias requeridas

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Nuevo directorio de trabajo
new_wd = "C:/Users/emili/Desktop/Rosario/Trabajo de grado/Crypto_DRL - Copy (2)/"
os.chdir(new_wd)

# Recarga directorios
%reload_ext autoreload
%autoreload 2

In [59]:
# Función para cargar datos de criptomonedas

from my_code.utils import load_multiple_prices

crypto_pairs = ['BTC/USDT','ETH/USDT','BNB/USDT',
                'XRP/USDT','SOL/USDT','ADA/USDT',
                'DOGE/USDT','TRX/USDT','LINK/USDT',
                'MATIC/USDT','DOT/USDT','LTC/USDT',
                'DAI/USDT','SHIB/USDT']

initial_year = 2023

crypto_data = load_multiple_prices(crypto_pairs, initial_year)

BTC/USDT downloaded from binance and stored at data/raw/binance-BTCUSDT-1h.pkl
Shape of BTC/USDT: (7433, 6)
ETH/USDT downloaded from binance and stored at data/raw/binance-ETHUSDT-1h.pkl
Shape of ETH/USDT: (7433, 6)
BNB/USDT downloaded from binance and stored at data/raw/binance-BNBUSDT-1h.pkl
Shape of BNB/USDT: (7433, 6)
XRP/USDT downloaded from binance and stored at data/raw/binance-XRPUSDT-1h.pkl
Shape of XRP/USDT: (7433, 6)
SOL/USDT downloaded from binance and stored at data/raw/binance-SOLUSDT-1h.pkl
Shape of SOL/USDT: (7433, 6)
ADA/USDT downloaded from binance and stored at data/raw/binance-ADAUSDT-1h.pkl
Shape of ADA/USDT: (7433, 6)
DOGE/USDT downloaded from binance and stored at data/raw/binance-DOGEUSDT-1h.pkl
Shape of DOGE/USDT: (7433, 6)
TRX/USDT downloaded from binance and stored at data/raw/binance-TRXUSDT-1h.pkl
Shape of TRX/USDT: (7433, 6)
LINK/USDT downloaded from binance and stored at data/raw/binance-LINKUSDT-1h.pkl
Shape of LINK/USDT: (7433, 6)
MATIC/USDT downloaded 

In [62]:
import numpy as np
import pandas as pd

def calculate_crypto_metrics(df):
    if df.empty:
        return {
            'Volatility': None,
            'Average Volume': None,
            'Average Range': None,
            'Average VWAP': None
        }

    # Calculate the logarithmic return
    df['log_return'] = np.log(df['close'] / df['close'].shift(1))
    
    # Calculate the metrics
    average_vwap = ((df['volume'] * (df['high'] + df['low'] + df['close']) / 3).cumsum() / df['volume'].cumsum())
    metrics = {
        'Volatility': df['log_return'].std(ddof=0) if not df['log_return'].empty else None,  # ddof=0 for population standard deviation
        'Average Volume': df['volume'].mean(),
        'Average Range': (df['high'] - df['low']).mean(),
        'Average VWAP': average_vwap.iloc[-1] if not average_vwap.empty else None
    }
    
    return metrics

# Iterate over each pair
all_metrics = {pair: calculate_crypto_metrics(crypto_data.get(pair, pd.DataFrame())) for pair in crypto_pairs}

# Sort the cryptocurrencies by our selection criteria
sorted_pairs = sorted(
    all_metrics.items(),
    key=lambda x: (
        x[1]['Volatility'] if x[1]['Volatility'] is not None else float('inf'),
        -x[1]['Average Volume'] if x[1]['Average Volume'] is not None else float('-inf'),
        -x[1]['Average Range'] if x[1]['Average Range'] is not None else float('-inf')
    )
)

# Select the top 5 cryptocurrencies based on the criteria
top_5_pairs = sorted_pairs[:min(5, len(sorted_pairs))]

# Convert this list of tuples into a DataFrame
df_metrics = pd.DataFrame.from_records(top_5_pairs, columns=['Pair', 'Metrics'])

# Expand the 'Metrics' column into separate columns
df_metrics_expanded = pd.concat([df_metrics.drop(['Metrics'], axis=1), df_metrics['Metrics'].apply(pd.Series)], axis=1)

# Format the DataFrame
formatted_df = df_metrics_expanded.style.format({
    "Volatility": "{:.4f}",
    "Average Volume": "{:,.0f}",
    "Average Range": "{:.4f}",
    "Average VWAP": "{:.2f}"
})

# To display the formatted DataFrame, you would normally return it or print it in a notebook.
# Here, we can just show the representation of the style object.
formatted_df


,Pair,Volatility,Average Volume,Average Range,Average VWAP
0,BTC/USDT,0.0046,"4,647",154.8934,24273.90
1,ETH/USDT,0.0049,"15,456",10.7991,1723.79
2,BNB/USDT,0.0049,"17,840",1.7173,267.89
3,TRX/USDT,0.0050,"14,034,343",0.0004,0.07
4,DOT/USDT,0.0065,"158,367",0.0475,5.55
